In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from config import GptConfig
from model import GPT
from dataclasses import asdict

torch.cuda.is_bf16_supported()

True

In [3]:
# from datasets import load_dataset
# import tiktoken
# import numpy as np

# repo_name = "Elriggs/openwebtext-100k"
# ds = load_dataset(repo_name, split='train')
# tokenizer = tiktoken.get_encoding('gpt2')
# eot = tokenizer.eot_token
# CONTEXT_LEN = 1024

# def tokenize(doc):
#     all_tokens = []
#     input_tokens = []
#     target = []
#     token_lists = tokenizer.encode_ordinary_batch(doc['text'])
#     for tokens in token_lists:
#         all_tokens.extend([eot] + tokens)
        
#     for i in range(0, len(all_tokens) - CONTEXT_LEN, CONTEXT_LEN):
#         input_tokens.append(all_tokens[i : i + CONTEXT_LEN])
#         target.append(all_tokens[i+1 :i + CONTEXT_LEN + 1])
        
#     return {'input': input_tokens, "labels": target}


# ds_map = ds.map(tokenize, batch_size=100000, batched=True, remove_columns=['text'])
# ds_map.save_to_disk('./pretrain_data/openwebtext_tokenized')

In [9]:
from config import GptConfig
from model import GPT
from utils import load_checkpoint
import torch
config = GptConfig(vocab_size=50304)

model = GPT(config).to(config.device)

model = torch.compile(model)

# model.save_pretrained(config=config, save_directory='checkpoints')

# model.from_pretrained('checkpoints')

In [4]:
# asdict(config)

In [2]:
from config import GptConfig
from model import GPT
from utils import load_checkpoint
import torch
config = GptConfig(vocab_size=50304)



# sample = torch.randint(1, 10, size=(1, 5)).to(config.device)
# target = torch.randint(1, 10, size=(1, 5)).to(config.device)

model = GPT(config).to(config.device)

model = torch.compile(model)
load_checkpoint(model)



64

In [6]:
# model.save_pretrained(config=config, save_directory='checkpoints')

model = model.from_pretrained('checkpoints')

In [ ]:
import inspect

In [9]:
print(f"total parameters in the model")
total_params = 0
for params in model.parameters():
    total_params+=params.numel()
print(total_params)

total parameters in the model
124507008


In [7]:
inputs = ['hai iam goint to'] * 2
model.generate(text=inputs, temperature=0.9, top_k=10)

['hai iam goint to you was the first and, and, and the a and not to the the and that with this.\n\n\n\n\n\n\n\n\n\nThe a the you to a a a a a a a is that to a to the is',
 'hai iam goint to be a the the the you of�\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThe a have is to the he are you was']

In [14]:
model = model.from_pretrained('checkpoints')

Some weights of GPT were not initialized from the model checkpoint at checkpoints and are newly initialized: ['final_layer.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from dataset import get_data_loader

In [ ]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')
loader = get_data_loader(tokenizer)

In [ ]:
for x, y in loader:
    print(x, y)
    break

tensor([[   12,   448,   547,  ...,  1867,   338,   534],
        [   40,  2937,    25,  ...,   339,   318, 10657],
        [   25,   198, 39461,  ...,   307,   354,  2903],
        ...,
        [  561,   407,   423,  ...,  8322,  9399,    25],
        [   12,  1820,    12,  ...,    13,   198,   198],
        [  910,     0,  8805,  ...,   815,   307,  6190]]) tensor([[  448,   547,   286,  ...,   338,   534,  1438],
        [ 2937,    25,   198,  ...,   318, 10657,    26],
        [  198, 39461,    11,  ...,   354,  2903,   606],
        ...,
        [  407,   423,   340,  ...,  9399,    25,   198],
        [ 1820,    12,    67,  ...,   198,   198, 18973],
        [    0,  8805,   321,  ...,   307,  6190,    25]])


In [19]:
from datasets import load_dataset, load_from_disk
data = load_from_disk('pretrain_data\openwebtext_tokenized')

<>:2: SyntaxWarning: invalid escape sequence '\o'
<>:2: SyntaxWarning: invalid escape sequence '\o'
C:\Users\VICTUS\AppData\Local\Temp\ipykernel_3152\705726110.py:2: SyntaxWarning: invalid escape sequence '\o'
  data = load_from_disk('pretrain_data\openwebtext_tokenized')


In [10]:
from torch.utils.data import DataLoader

In [42]:
data.set_format('torch')

In [45]:
data_l = DataLoader(data, batch_size=5, shuffle=True)

In [46]:
for i in data_l:
    break

In [47]:
i

{'input': tensor([[ 6051,   264,    73,  ...,  3068,  4688,  3651],
         [ 2087,    13,   198,  ...,   881,   284,  2897],
         [ 1081,  4302,  7443,  ...,   405, 20872, 44824],
         [44767, 47031,   284,  ...,  5479,   355,   890],
         [   82,   564,   246,  ...,   198,  2396,    11]]),
 'labels': tensor([[  264,    73, 11033,  ...,  4688,  3651,   925],
         [   13,   198,   198,  ...,   284,  2897,     0],
         [ 4302,  7443,  4710,  ..., 20872, 44824,   272],
         [47031,   284,  1241,  ...,   355,   890,   355],
         [  564,   246,  1462,  ...,  2396,    11,   477]])}

In [9]:
# from torch.utils.data import DataLoader

In [3]:
params = {n: p for n, p in model.named_parameters()}
params = {n: p for n, p in params.items() if p.requires_grad}

decay_params = [p for n, p in params.items() if p.dim() >= 2]
nodecay_params = [p for n, p in params.items() if p.dim() < 2]

param_grps = [
    {'params': decay_params, 'weight_decay': .1},
    {'params': nodecay_params, 'weight_decay': 0.0}
]

num_decay_params = sum(p.numel() for p in decay_params)
num_nodecay_params = sum(p.numel() for p in nodecay_params)

print(f"total num of decay parms :{num_decay_params} with num tensor of {len(decay_params)}")
print(f"total num of no decay parms :{num_nodecay_params} with num tensor of {len(nodecay_params)}")
import inspect;
fused_available = "fused" in inspect.signature(torch.optim.AdamW).parameters
use_fused = fused_available and 'cuda' in model.config.device
print(f'Using fused: {use_fused}')

total num of decay parms :124354560 with num tensor of 74
total num of no decay parms :152448 with num tensor of 98
Using fused: False


In [14]:
162988032

162988032